<a href="https://colab.research.google.com/github/kenny08gt/Richter-s-Predictor-Modeling-Earthquake-Damage-DS/blob/main/Richter's_Predictor_Modeling_Earthquake_Damage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow import keras

We're trying to predict the ordinal variable damage_grade, which represents a level of damage to the building that was hit by the earthquake. There are 3 grades of the damage:

1 represents low damage
2 represents a medium amount of damage
3 represents almost complete destruction

In [22]:
import pandas as pd 

#building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,ground_floor_type,other_floor_type,position,plan_configuration,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,legal_ownership_status,count_families,has_secondary_use,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
df = pd.read_csv("/content/drive/MyDrive/Data science/DataDriven/Richters predictor/train_values.csv")
labels = pd.read_csv("/content/drive/MyDrive/Data science/DataDriven/Richters predictor/train_labels.csv")
df = pd.merge(df, labels, on=['building_id'])

print(df.head(10))

   building_id  geo_level_1_id  ...  has_secondary_use_other  damage_grade
0       802906               6  ...                        0             3
1        28830               8  ...                        0             2
2        94947              21  ...                        0             3
3       590882              22  ...                        0             2
4       201944              11  ...                        0             3
5       333020               8  ...                        0             2
6       728451               9  ...                        0             3
7       475515              20  ...                        0             1
8       441126               0  ...                        0             2
9       989500              26  ...                        0             1

[10 rows x 40 columns]


In [39]:
df.dtypes

building_id                                int64
geo_level_1_id                             int64
geo_level_2_id                             int64
geo_level_3_id                             int64
count_floors_pre_eq                        int64
age                                        int64
area_percentage                            int64
height_percentage                          int64
land_surface_condition                    object
foundation_type                           object
roof_type                                 object
ground_floor_type                         object
other_floor_type                          object
position                                  object
plan_configuration                        object
has_superstructure_adobe_mud               int64
has_superstructure_mud_mortar_stone        int64
has_superstructure_stone_flag              int64
has_superstructure_cement_mortar_stone     int64
has_superstructure_mud_mortar_brick        int64
has_superstructure_c

In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 260601 entries, 0 to 260600
Data columns (total 40 columns):
 #   Column                                  Non-Null Count   Dtype 
---  ------                                  --------------   ----- 
 0   building_id                             260601 non-null  int64 
 1   geo_level_1_id                          260601 non-null  int64 
 2   geo_level_2_id                          260601 non-null  int64 
 3   geo_level_3_id                          260601 non-null  int64 
 4   count_floors_pre_eq                     260601 non-null  int64 
 5   age                                     260601 non-null  int64 
 6   area_percentage                         260601 non-null  int64 
 7   height_percentage                       260601 non-null  int64 
 8   land_surface_condition                  260601 non-null  object
 9   foundation_type                         260601 non-null  object
 10  roof_type                               260601 non-null 

In [12]:
corr = df.corr()
corr.style.background_gradient(cmap='coolwarm', axis=None)

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,count_families,has_secondary_use,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,damage_grade
building_id,1.000000,-0.002850,0.000347,-0.000393,-0.000654,-0.001476,-0.002070,0.000096,-0.000307,0.002423,0.001443,-0.000827,-0.003768,0.001933,0.001967,0.001266,0.002472,-0.002259,0.002171,-0.000599,0.002606,0.002631,0.001934,-0.002152,0.000706,-0.000362,0.002348,-0.000374,0.000538,-0.003116,-0.002295,0.001063
geo_level_1_id,-0.002850,1.000000,-0.061405,0.002718,-0.089364,-0.003908,0.071158,-0.063474,-0.018245,-0.152038,0.007677,0.023656,-0.028854,0.134121,0.144985,0.074169,0.016454,0.015767,0.022670,0.035108,-0.010428,-0.019786,0.001911,0.023523,0.003700,0.002977,0.002655,-0.002303,0.001060,0.000523,-0.017992,-0.072347
geo_level_2_id,0.000347,-0.061405,1.000000,0.000921,0.047730,0.012594,-0.049443,0.035516,0.015833,0.076491,-0.008743,-0.009122,0.014246,-0.079875,-0.023927,-0.032588,-0.038173,-0.036074,-0.020144,-0.012439,-0.001519,0.017522,-0.008439,-0.030704,-0.004840,-0.004856,0.000687,-0.000757,-0.000152,0.001926,-0.013068,0.043161
geo_level_3_id,-0.000393,0.002718,0.000921,1.000000,-0.021646,-0.006385,-0.005643,-0.024507,-0.015732,0.026294,0.019579,0.005966,-0.023278,-0.005116,-0.003916,0.006143,0.002265,-0.012420,-0.010604,-0.002479,0.007375,0.015995,-0.002001,-0.007356,-0.007058,-0.004373,-0.000862,-0.002632,-0.000943,0.000269,-0.002463,0.007932
count_floors_pre_eq,-0.000654,-0.089364,0.047730,-0.021646,1.000000,0.086668,0.101071,0.772734,0.174852,-0.027116,-0.041058,-0.011329,0.257279,-0.085821,-0.056637,-0.070487,0.007448,0.049000,-0.029888,0.086586,0.052125,-0.005294,0.077120,0.035425,0.016384,0.008833,-0.002611,0.006786,0.009639,0.003939,-0.002073,0.122308
age,-0.001476,-0.003908,0.012594,-0.006385,0.086668,1.000000,-0.004323,0.061074,0.068032,0.001321,0.009129,-0.012128,0.079525,-0.036992,0.005855,-0.008374,-0.025431,-0.025552,-0.001520,0.005309,-0.008788,-0.002194,-0.010021,0.001193,-0.004189,-0.003514,-0.003658,-0.002169,-0.001764,-0.001195,-0.004534,0.029273
area_percentage,-0.002070,0.071158,-0.049443,-0.005643,0.101071,-0.004323,1.000000,0.196645,0.026287,-0.225541,-0.000473,0.073701,0.053362,0.210800,-0.053965,-0.031641,0.185047,0.224647,0.007928,0.088630,0.122401,-0.016478,0.159885,0.105983,0.052212,0.050164,0.019421,0.015109,0.015290,0.004983,0.013111,-0.125221
height_percentage,0.000096,-0.063474,0.035516,-0.024507,0.772734,0.061074,0.196645,1.000000,0.149725,-0.106573,-0.020260,0.012960,0.209098,0.001698,-0.052402,-0.063342,0.086228,0.129698,-0.019266,0.064316,0.091780,-0.005390,0.123551,0.068909,0.031366,0.020032,0.001946,0.011192,0.014660,0.004048,0.005397,0.048130
has_superstructure_adobe_mud,-0.000307,-0.018245,0.015833,-0.015732,0.174852,0.068032,0.026287,0.149725,1.000000,-0.306861,0.006962,-0.014554,0.314979,-0.037549,0.011694,0.011528,-0.036830,-0.036782,0.057545,0.033068,-0.013300,-0.003917,-0.012642,-0.003935,-0.004281,-0.002369,0.001762,-0.003292,-0.002648,-0.001493,-0.010074,0.055314
has_superstructure_mud_mortar_stone,0.002423,-0.152038,0.076491,0.026294,-0.027116,0.001321,-0.225541,-0.106573,-0.306861,1.000000,-0.034046,-0.104288,-0.376208,-0.470715,-0.040177,-0.054657,-0.222366,-0.224509,-0.041705,0.001576,-0.087520,0.058261,-0.159532,-0.117948,-0.036064,-0.023070,-0.025507,-0.008763,-0.011904,-0.003380,0.005628

There seems to be no directly correlated variable with damage



In [23]:
df.describe()


,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,count_families,has_secondary_use,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,damage_grade
count,2.606010e+05,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000
mean,5.256755e+05,13.900353,701.074685,6257.876148,2.129723,26.535029,8.018051,5.434365,0.088645,0.761935,0.034332,0.018235,0.068154,0.075268,0.254988,0.085011,0.042590,0.015859,0.014985,0.983949,0.111880,0.064378,0.033626,0.008101,0.000940,0.000361,0.001071,0.000188,0.000146,0.000088,0.005119,2.238272
std,3.045450e+05,8.033617,412.710734,3646.369645,0.727665,73.565937,4.392231,1.918418,0.284231,0.425900,0.182081,0.133800,0.252010,0.263824,0.435855,0.278899,0.201931,0.124932,0.121491,0.418389,0.315219,0.245426,0.180265,0.089638,0.030647,0.018989,0.032703,0.013711,0.012075,0.009394,0.071364,0.611814
min,4.000000e+00,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,2.611900e+05,7.000000,350.000000,3073.000000,2.000000,10.000000,5.000000,4.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000
50%,5.257570e+05,12.000000,702.000000,6270.000000,2.000000,15.000000,7.000000,5.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000
75%,7.897620e+05,21.000000,1050.000000,9412.000000,2.000000,30.000000,9.000000,6.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000
max,1.052934e+06,30.000000,1427.000000,12567.000000,9.000000,995.000000,100.000000,32.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,9.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,3.000000


In [59]:
training_data = df.copy()
del training_data['building_id']
del training_data['geo_level_3_id']
del training_data['has_secondary_use_health_post']
del training_data['has_secondary_use_gov_office']
del training_data['has_secondary_use_use_police']
del training_data['has_secondary_use_other']

training_data["geo_level_1_id"] /= 30
training_data['geo_level_2_id'] /= 1427
training_data['count_floors_pre_eq'] /= 9
training_data['age'] /= 995
training_data['area_percentage'] /= 100
training_data['height_percentage'] /= 32
#n, o, t.
training_data['land_surface_condition'] = training_data['land_surface_condition'].str.replace('n', '0').str.replace('o', '1').str.replace('t', '2')
training_data['land_surface_condition'] = pd.to_numeric(training_data['land_surface_condition'], errors='coerce').astype('Int64')
#n, q, x.
training_data['roof_type'] = training_data['roof_type'].str.replace('n', '0').str.replace('q', '1').str.replace('x', '2')
training_data['roof_type'] = pd.to_numeric(training_data['roof_type'], errors='coerce').astype('Int64')
#h, i, r, u, w
training_data['foundation_type'] = training_data['foundation_type'].str.replace('h', '0').str.replace('i', '1').str.replace('r', '2').str.replace('u', '3').str.replace('w', '4')
training_data['foundation_type'] = pd.to_numeric(training_data['foundation_type'], errors='coerce').astype('Int64')
#f, m, v, x, z.
training_data['ground_floor_type'] = training_data['ground_floor_type'].str.replace('f', '0').str.replace('m', '1').str.replace('v', '2').str.replace('x', '3').str.replace('z', '4')
training_data['ground_floor_type'] = pd.to_numeric(training_data['ground_floor_type'], errors='coerce').astype('Int64')
#j, q, s, x.
training_data['other_floor_type'] = training_data['other_floor_type'].str.replace('j', '0').str.replace('q', '1').str.replace('s', '2').str.replace('x', '3')
training_data['other_floor_type'] = pd.to_numeric(training_data['other_floor_type'], errors='coerce').astype('Int64')
#j, o, s, t.
training_data['position'] = training_data['position'].str.replace('j', '0').str.replace('o', '1').str.replace('s', '2').str.replace('t', '3')
training_data['position'] = pd.to_numeric(training_data['position'], errors='coerce').astype('Int64')
#a, c, d, f, m, n, o, q, s, u.
training_data['plan_configuration'] = training_data['plan_configuration'].str.replace('a', '0').str.replace('c', '1').str.replace('d', '2').str.replace('f', '3').str.replace('m', '4').str.replace('n', '5').str.replace('o', '6').str.replace('q', '7').str.replace('s', '8').str.replace('u', '9')
training_data['plan_configuration'] = pd.to_numeric(training_data['plan_configuration'], errors='coerce').astype('Int64')
#a, r, v, w.
training_data['legal_ownership_status'] = training_data['legal_ownership_status'].str.replace('a', '0').str.replace('r', '1').str.replace('v', '2').str.replace('w', '3')
training_data['legal_ownership_status'] = pd.to_numeric(training_data['legal_ownership_status'], errors='coerce').astype('Int64')


training_data['count_families'] /= 9
training_data.describe()

,geo_level_1_id,geo_level_2_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,ground_floor_type,other_floor_type,position,plan_configuration,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,legal_ownership_status,count_families,has_secondary_use,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,damage_grade
count,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000
mean,0.463345,0.491293,0.236636,0.026668,0.080181,0.169824,1.695427,2.119036,0.360482,0.492481,1.226711,2.053718,2.215325,0.088645,0.761935,0.034332,0.018235,0.068154,0.075268,0.254988,0.085011,0.042590,0.015859,0.014985,1.962318,0.109328,0.111880,0.064378,0.033626,0.008101,0.000940,0.000361,0.001071,2.238272
std,0.267787,0.289216,0.080852,0.073936,0.043922,0.059951,0.696040,0.579154,0.595595,1.028436,0.902518,0.611996,1.118290,0.284231,0.425900,0.182081,0.133800,0.252010,0.263824,0.435855,0.278899,0.201931,0.124932,0.121491,0.314817,0.046488,0.315219,0.245426,0.180265,0.089638,0.030647,0.018989,0.032703,0.611814
min,0.000000,0.000000,0.111111,0.000000,0.010000,0.062500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,0.233333,0.245270,0.222222,0.010050,0.050000,0.125000,2.000000,2.000000,0.000000,0.000000,1.000000,2.000000,2.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000
50%,0.400000,0.491941,0.222222,0.015075,0.070000,0.156250,2.000000,2.000000,0.000000,0.000000,1.000000,2.000000,2.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000
75%,0.700000,0.735809,0.222222,0.030151,0.090000,0.187500,2.000000,2.000000,1.000000,0.000000,1.000000,2.000000,2.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,4.000000,2.000000,4.000000,3.000000,3.000000,9.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,3.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,3.000000


In [60]:
training_data.dtypes

geo_level_1_id                            float64
geo_level_2_id                            float64
count_floors_pre_eq                       float64
age                                       float64
area_percentage                           float64
height_percentage                         float64
land_surface_condition                      Int64
foundation_type                             Int64
roof_type                                   Int64
ground_floor_type                           Int64
other_floor_type                            Int64
position                                    Int64
plan_configuration                          Int64
has_superstructure_adobe_mud                int64
has_superstructure_mud_mortar_stone         int64
has_superstructure_stone_flag               int64
has_superstructure_cement_mortar_stone      int64
has_superstructure_mud_mortar_brick         int64
has_superstructure_cement_mortar_brick      int64
has_superstructure_timber                   int64


In [61]:
corr = training_data.corr()
corr.style.background_gradient(cmap='coolwarm', axis=None)

,geo_level_1_id,geo_level_2_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,ground_floor_type,other_floor_type,position,plan_configuration,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,legal_ownership_status,count_families,has_secondary_use,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,damage_grade
geo_level_1_id,1.000000,-0.061405,-0.089364,-0.003908,0.071158,-0.063474,0.030434,0.197030,0.040651,0.054917,-0.001010,-0.010528,0.032916,-0.018245,-0.152038,0.007677,0.023656,-0.028854,0.134121,0.144985,0.074169,0.016454,0.015767,0.022670,-0.092157,0.035108,-0.010428,-0.019786,0.001911,0.023523,0.003700,0.002977,0.002655,-0.072347
geo_level_2_id,-0.061405,1.000000,0.047730,0.012594,-0.049443,0.035516,-0.034365,-0.034404,-0.090820,-0.093396,-0.015013,0.009542,-0.023195,0.015833,0.076491,-0.008743,-0.009122,0.014246,-0.079875,-0.023927,-0.032588,-0.038173,-0.036074,-0.020144,0.034451,-0.012439,-0.001519,0.017522,-0.008439,-0.030704,-0.004840,-0.004856,0.000687,0.043161
count_floors_pre_eq,-0.089364,0.047730,1.000000,0.086668,0.101071,0.772734,0.043832,-0.149130,0.091733,-0.048276,0.369606,-0.176834,-0.025296,0.174852,-0.027116,-0.041058,-0.011329,0.257279,-0.085821,-0.056637,-0.070487,0.007448,0.049000,-0.029888,0.092764,0.086586,0.052125,-0.005294,0.077120,0.035425,0.016384,0.008833,-0.002611,0.122308
age,-0.003908,0.012594,0.086668,1.000000,-0.004323,0.061074,0.015866,-0.031302,-0.033532,-0.035342,0.010692,-0.048256,-0.004309,0.068032,0.001321,0.009129,-0.012128,0.079525,-0.036992,0.005855,-0.008374,-0.025431,-0.025552,-0.001520,0.023291,0.005309,-0.008788,-0.002194,-0.010021,0.001193,-0.004189,-0.003514,-0.003658,0.029273
area_percentage,0.071158,-0.049443,0.101071,-0.004323,1.000000,0.196645,0.027887,-0.103044,0.259252,0.204712,0.109258,-0.040626,0.087245,0.026287,-0.225541,-0.000473,0.073701,0.053362,0.210800,-0.053965,-0.031641,0.185047,0.224647,0.007928,0.028479,0.088630,0.122401,-0.016478,0.159885,0.105983,0.052212,0.050164,0.019421,-0.125221
height_percentage,-0.063474,0.035516,0.772734,0.061074,0.196645,1.000000,0.022876,-0.149241,0.165649,0.041635,0.348153,-0.148047,-0.005858,0.149725,-0.106573,-0.020260,0.012960,0.209098,0.001698,-0.052402,-0.063342,0.086228,0.129698,-0.019266,0.074417,0.064316,0.091780,-0.005390,0.123551,0.068909,0.031366,0.020032,0.001946,0.048130
land_surface_condition,0.030434,-0.034365,0.043832,0.015866,0.027887,0.022876,1.000000,-0.009165,0.054031,0.016903,-0.030527,-0.025109,0.005437,0.010201,-0.079695,-0.045326,-0.011771,0.058683,0.058912,-0.041338,-0.027842,0.006346,0.027483,-0.035197,0.012540,0.008894,0.003745,-0.006204,0.009847,0.008792,0.003889,0.004538,0.002525,-0.012218
foundation_type,0.197030,-0.034404,-0.149130,-0.031302,-0.103044,-0.149241,-0.009165,1.000000,-0.074317,-0.039513,-0.021252,0.008386,0.025207,-0.049766,-0.155522,-0.047451,0.040425,-0.032870,0.087475,0.265907,0.242956,-0.162128,-0.199355,0.003281,-0.199372,-0.009598,-0.051616,0.006990,-0.069214,-0.049471,-0.021222,-0.006347,0.004415,-0.100601
roof_type,0.040651,-0.090820,0.091733,-0.033532,0.259252,0.165649,0.054031,-0.074317,1.000000,0.263619,0.048931,-0.041272,0.057572,-0.072842,-0.319488,-0.033446,0.045215,-0.011186,0.288584,-0.053304,-0.011784,0.320016,0.329077,-0.020241,-0.019996,0.000122,0.086249,-0.056567,0.150329,0.129091,0.046603,0.021075,0.010722,-0.173890
ground_floor_type,0.054917,-0.093396,-0.048276,-0.035342,0.204712,0.041635,0.016903,-0.039513,0.263619,1.000000,0.010698,-0.046715,0.065195,-0.050577,-0.304966,0.026348,0.121578,0.03359

In [69]:
training_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 260601 entries, 0 to 260600
Data columns (total 34 columns):
 #   Column                                  Non-Null Count   Dtype  
---  ------                                  --------------   -----  
 0   geo_level_1_id                          260601 non-null  float64
 1   geo_level_2_id                          260601 non-null  float64
 2   count_floors_pre_eq                     260601 non-null  float64
 3   age                                     260601 non-null  float64
 4   area_percentage                         260601 non-null  float64
 5   height_percentage                       260601 non-null  float64
 6   land_surface_condition                  260601 non-null  Int64  
 7   foundation_type                         260601 non-null  Int64  
 8   roof_type                               260601 non-null  Int64  
 9   ground_floor_type                       260601 non-null  Int64  
 10  other_floor_type                        2606

In [92]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

X = training_data.drop('damage_grade', axis=1)
y = training_data['damage_grade']

y = to_categorical(y)
print(y)

random_state = 42
batch_size = 250
validation_split = 0.2

X_train, X_test, y_train, y_test = train_test_split(np.asarray(X).astype('float32'), y, test_size=0.33, random_state=random_state)



[[0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]
 ...
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]


In [101]:
n_epochs = 500
# Create the model
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(32, activation='relu', input_dim=33))
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dense(512, activation='relu'))
model.add(tf.keras.layers.Dense(8, activation='relu'))
model.add(tf.keras.layers.Dense(4, activation='sigmoid'))

model.summary()


# Compile the model
model.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.SGD(learning_rate=1e-3, momentum=0.9),
              metrics=['accuracy'])

# Fit data to model
history = model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=n_epochs,
          validation_split=validation_split)

# model.compile(loss = 'categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

# history = model.fit(train_generator, epochs=25, steps_per_epoch=20, validation_data = validation_generator, verbose = 1, validation_steps=3)

Model: "sequential_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_82 (Dense)             (None, 32)                1088      
_________________________________________________________________
dense_83 (Dense)             (None, 128)               4224      
_________________________________________________________________
dense_84 (Dense)             (None, 512)               66048     
_________________________________________________________________
dense_85 (Dense)             (None, 8)                 4104      
_________________________________________________________________
dense_86 (Dense)             (None, 4)                 36        
Total params: 75,500
Trainable params: 75,500
Non-trainable params: 0
_________________________________________________________________
Epoch 1/500
559/559 [==============================] - 3s 5ms/step - loss: 0.9342 - accuracy: 0.5669 - val_loss: 0.

|lr|epochs|acc|val_Acc|conver epoch|momentum|
|---|---|---|---|---|---|
|1e-6|5|0.5685|0.5706|4|0.9|
|1e-3|100|0.6371|0.6365|100|0.9|
|1e-3|500|0.6881|0.6731|500|0.9|

In [106]:
n_epochs = 800
# Create the model
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(32, activation='relu', input_dim=33))
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dense(512, activation='relu'))
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dense(4, activation='sigmoid'))

model.summary()


# Compile the model
model.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.SGD(learning_rate=1e-3, momentum=0.9),
              metrics=['accuracy'])

# Fit data to model
history = model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=n_epochs,
          validation_split=validation_split)

# model.compile(loss = 'categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

# history = model.fit(train_generator, epochs=25, steps_per_epoch=20, validation_data = validation_generator, verbose = 1, validation_steps=3)

Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_99 (Dense)             (None, 32)                1088      
_________________________________________________________________
dense_100 (Dense)            (None, 128)               4224      
_________________________________________________________________
dense_101 (Dense)            (None, 512)               66048     
_________________________________________________________________
dense_102 (Dense)            (None, 256)               131328    
_________________________________________________________________
dense_103 (Dense)            (None, 128)               32896     
_________________________________________________________________
dense_104 (Dense)            (None, 4)                 516       
Total params: 236,100
Trainable params: 236,100
Non-trainable params: 0
_______________________________________________

|lr|epochs|acc|val_Acc|conver epoch|momentum|
|---|---|---|---|---|---|
|1e-3|100|0.6499|0.6470|100|0.9|
|1e-3|500|0.7044|0.6780|100|0.9|
|1e-3|800|0.7232|0.6746|100|0.9|

In [114]:
selected_features = ['foundation_type', 
                     'area_percentage', 
                     'height_percentage',
                     'count_floors_pre_eq',
                     'land_surface_condition',
                     'has_superstructure_cement_mortar_stone']
X2 = X[selected_features]
X2.info()
X_train, X_test, y_train, y_test = train_test_split(np.asarray(X2).astype('float32'), y, test_size=0.33, random_state=random_state)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 260601 entries, 0 to 260600
Data columns (total 6 columns):
 #   Column                                  Non-Null Count   Dtype  
---  ------                                  --------------   -----  
 0   foundation_type                         260601 non-null  Int64  
 1   area_percentage                         260601 non-null  float64
 2   height_percentage                       260601 non-null  float64
 3   count_floors_pre_eq                     260601 non-null  float64
 4   land_surface_condition                  260601 non-null  Int64  
 5   has_superstructure_cement_mortar_stone  260601 non-null  int64  
dtypes: Int64(2), float64(3), int64(1)
memory usage: 24.4 MB


In [119]:
n_epochs = 100
# Create the model
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(32, activation='relu', input_dim=6))
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dense(512, activation='relu'))
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dense(4, activation='sigmoid'))

model.summary()


# Compile the model
model.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.SGD(learning_rate=1e-1, momentum=0.6),
              metrics=['accuracy'])

# Fit data to model
history = model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=n_epochs,
          validation_split=validation_split)

# model.compile(loss = 'categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

# history = model.fit(train_generator, epochs=25, steps_per_epoch=20, validation_data = validation_generator, verbose = 1, validation_steps=3)

Model: "sequential_29"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_141 (Dense)            (None, 32)                224       
_________________________________________________________________
dense_142 (Dense)            (None, 128)               4224      
_________________________________________________________________
dense_143 (Dense)            (None, 512)               66048     
_________________________________________________________________
dense_144 (Dense)            (None, 256)               131328    
_________________________________________________________________
dense_145 (Dense)            (None, 128)               32896     
_________________________________________________________________
dense_146 (Dense)            (None, 4)                 516       
Total params: 235,236
Trainable params: 235,236
Non-trainable params: 0
_______________________________________________

KeyboardInterrupt: ignored